## Contents
### 0. Importing Libraries and dataset
### 1. Preprocessing
### 2. Doc2Vec using Law2Vec 100d
### 3. Doc2Vec using Law2Vec 00d
### 4. Doc2Vec using DMPV
### 5. Doc2Vec using DBOW

##### -Written by Sparsh Goil, DA-IICT, Gandhinagar, India 

# 

In [1]:
import string as stringl
import re
import logging
import gensim.models as g
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import remove_stopwords
from os import walk
from nltk import word_tokenize,sent_tokenize
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer

[nltk_data] Downloading package stopwords to /home/spaggy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/spaggy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Load Dataset-CaseDocs
Dataset= "/home/spaggy/Documents/NLP Project 2020/AILA-dataset/dataset/AILA-data/Object_casedocs"
Filenames=[]
labels=[]
for (dirpath, dirnames, filenames) in walk(Dataset):
        Filenames.extend(filenames)
        for filename in filenames:
            labels.append(filename[:-4])
        break

Documents=[]
for doc in Filenames:
    file=Dataset +"//" + doc
    file=open(file,'r')
    text=file.read()
    Documents.append(text)
    file.close()
    
#Load Queries
query_doc="/home/spaggy/Documents/NLP Project 2020/AILA-dataset/dataset/AILA-data/Query_doc.txt"
file = open(query_doc,'r')
text=file.read()
file.close()
queries=text.split("\n")
queries=queries[:-1]
j=0
for query in queries:
    query_name="AILA"+str(j+1)
    labels.append(query_name)
    for i in range(len(query)):
        if(query[i]=='|' and query[i+1]=='|'):
            queries[j]=queries[j][i+2:]
            j+=1
            break


## 1. Preprocessing

In [3]:
#Data Preprocessing
stop_words = set(stopwords.words('english')) 
stemmer=PorterStemmer()
lemmatizer=WordNetLemmatizer()
def text_processing(string,type):
    if type=="case docs":
        string=sent_tokenize(string)
    for i in range(len(string)):
        string[i]=string[i].lower()
        string[i]=re.sub(r'\d+', '', string[i])
        string[i]=string[i].translate(stringl.maketrans("",""),stringl.punctuation)
        string[i]=word_tokenize(string[i])

        string[i]=[word for word in string[i] if not word in stop_words]
        for j in range(len(string[i])):
            string[i][j]=stemmer.stem(string[i][j])
            string[i][j]=lemmatizer.lemmatize(string[i][j])
    f_string=[]
    for i in range(len(string)):
        for j in range(len(string[i])):
            f_string.append(string[i][j])
    return f_string

In [4]:
for i in range(len(Documents)):
    Documents[i]=text_processing(Documents[i],"case docs")
for i in range(len(queries)):
    queries[i]=text_processing([queries[i]],"query docs")
Documents_train=Documents+queries[0:10]
Documents_train = [TaggedDocument(doc, [label]) for label, doc in zip(labels,Documents_train)]

## 2. Doc2Vec using Law2Vec100d

## Run1

In [36]:
Documents_train=Documents+queries
Documents_train = [TaggedDocument(doc, [label]) for label, doc in zip(labels,Documents_train)]

In [16]:
saved_path = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec100d_run1.bin" #output model
pretrained_emb = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/law2vec embeddings/Law2Vec.100d.txt" #pretrained word embeddings
output_file="/home/spaggy/Documents/NLP Project 2020/AILA-dataset/Evaluation_061/evalutaion_file_law2vec100d_run1.txt" #Predicted output

In [40]:
#doc2vec parameters
vector_size = 100
window_size = 20
min_count = 1
sampling_threshold = 1e-6
negative_size = 5
train_epoch =50
dm = 1 #0 = dbow; 1 = dmpv
worker_count = 2 #number of parallel processes


In [41]:
#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#train doc2vec model
#model = g.Doc2Vec(Documents_train, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0,pretrained_emb=pretrained_emb, dm=1, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)
model = g.Doc2Vec(Documents_train, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm,pretrained_emb=pretrained_emb, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)

2020-05-04 19:08:17,257 : INFO : using concatenative 4100-dimensional layer1
2020-05-04 19:08:17,262 : INFO : collecting all words and their counts
2020-05-04 19:08:17,265 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-04 19:08:18,045 : INFO : collected 56521 word types and 2964 unique tags from a corpus of 2964 examples and 4871166 words
2020-05-04 19:08:18,047 : INFO : Loading a fresh vocabulary
2020-05-04 19:08:18,322 : INFO : effective_min_count=1 retains 56521 unique words (100% of original 56521, drops 0)
2020-05-04 19:08:18,323 : INFO : effective_min_count=1 leaves 4871166 word corpus (100% of original 4871166, drops 0)
2020-05-04 19:08:18,445 : INFO : deleting the raw counts dictionary of 56521 items
2020-05-04 19:08:18,449 : INFO : sample=1e-06 downsamples 10670 most-common words
2020-05-04 19:08:18,450 : INFO : downsampling leaves estimated 468577 word corpus (9.6% of prior 4871166)
2020-05-04 19:08:18,591 : INFO : estimated required m

2020-05-04 19:09:11,396 : INFO : EPOCH 6 - PROGRESS: at 19.53% examples, 46069 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:09:12,405 : INFO : EPOCH 6 - PROGRESS: at 30.36% examples, 47588 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:09:13,415 : INFO : EPOCH 6 - PROGRESS: at 40.25% examples, 47340 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:09:14,428 : INFO : EPOCH 6 - PROGRESS: at 50.71% examples, 47413 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:09:15,431 : INFO : EPOCH 6 - PROGRESS: at 61.81% examples, 47925 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:09:16,432 : INFO : EPOCH 6 - PROGRESS: at 71.90% examples, 48366 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:09:17,470 : INFO : EPOCH 6 - PROGRESS: at 82.19% examples, 48495 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:09:18,480 : INFO : EPOCH 6 - PROGRESS: at 91.16% examples, 48046 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:09:19,157 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19

2020-05-04 19:10:11,204 : INFO : EPOCH 12 - PROGRESS: at 31.11% examples, 48721 words/s, in_qsize 4, out_qsize 1
2020-05-04 19:10:12,216 : INFO : EPOCH 12 - PROGRESS: at 40.69% examples, 47571 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:10:13,217 : INFO : EPOCH 12 - PROGRESS: at 49.56% examples, 46274 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:10:14,236 : INFO : EPOCH 12 - PROGRESS: at 60.19% examples, 46404 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:10:15,271 : INFO : EPOCH 12 - PROGRESS: at 69.16% examples, 46348 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:10:16,282 : INFO : EPOCH 12 - PROGRESS: at 77.02% examples, 45044 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:10:17,296 : INFO : EPOCH 12 - PROGRESS: at 86.13% examples, 45159 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:10:18,298 : INFO : EPOCH 12 - PROGRESS: at 96.66% examples, 45740 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:10:18,463 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-

2020-05-04 19:11:10,182 : INFO : EPOCH 18 - PROGRESS: at 30.74% examples, 48002 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:11:11,183 : INFO : EPOCH 18 - PROGRESS: at 42.78% examples, 49632 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:11:12,193 : INFO : EPOCH 18 - PROGRESS: at 53.51% examples, 50161 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:11:13,200 : INFO : EPOCH 18 - PROGRESS: at 64.81% examples, 50553 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:11:14,222 : INFO : EPOCH 18 - PROGRESS: at 76.15% examples, 51031 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:11:15,227 : INFO : EPOCH 18 - PROGRESS: at 86.57% examples, 51476 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:11:16,227 : INFO : EPOCH 18 - PROGRESS: at 97.71% examples, 51537 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:11:16,266 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:11:16,275 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:11:16,276 : I

2020-05-04 19:12:10,169 : INFO : EPOCH 24 - PROGRESS: at 79.12% examples, 53465 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:12:11,182 : INFO : EPOCH 24 - PROGRESS: at 90.01% examples, 53600 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:12:11,883 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:12:11,891 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:12:11,892 : INFO : EPOCH - 24 : training on 4871166 raw words (472019 effective words) took 8.8s, 53779 effective words/s
2020-05-04 19:12:12,905 : INFO : EPOCH 25 - PROGRESS: at 12.01% examples, 54715 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:12:13,929 : INFO : EPOCH 25 - PROGRESS: at 23.11% examples, 55025 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:12:14,934 : INFO : EPOCH 25 - PROGRESS: at 34.85% examples, 55188 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:12:15,937 : INFO : EPOCH 25 - PROGRESS: at 47.20% examples, 54703 words/s, in_qsize 4, out_qsize 0
2020

2020-05-04 19:13:08,310 : INFO : EPOCH 31 - PROGRESS: at 47.03% examples, 54620 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:13:09,325 : INFO : EPOCH 31 - PROGRESS: at 55.67% examples, 52244 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:13:10,334 : INFO : EPOCH 31 - PROGRESS: at 66.13% examples, 51720 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:13:11,352 : INFO : EPOCH 31 - PROGRESS: at 74.33% examples, 49871 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:13:12,366 : INFO : EPOCH 31 - PROGRESS: at 83.33% examples, 49345 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:13:13,367 : INFO : EPOCH 31 - PROGRESS: at 93.72% examples, 49497 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:13:13,774 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:13:13,785 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:13:13,787 : INFO : EPOCH - 31 : training on 4871166 raw words (471337 effective words) took 9.5s, 49566 effective words/s
2020

2020-05-04 19:14:08,084 : INFO : EPOCH 38 - PROGRESS: at 10.63% examples, 48893 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:14:09,092 : INFO : EPOCH 38 - PROGRESS: at 20.24% examples, 48748 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:14:10,095 : INFO : EPOCH 38 - PROGRESS: at 31.65% examples, 49791 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:14:11,097 : INFO : EPOCH 38 - PROGRESS: at 42.04% examples, 49266 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:14:12,104 : INFO : EPOCH 38 - PROGRESS: at 52.90% examples, 50012 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:14:13,112 : INFO : EPOCH 38 - PROGRESS: at 64.30% examples, 50260 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:14:14,136 : INFO : EPOCH 38 - PROGRESS: at 75.17% examples, 50503 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:14:15,148 : INFO : EPOCH 38 - PROGRESS: at 85.93% examples, 50924 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:14:16,185 : INFO : EPOCH 38 - PROGRESS: at 96.93% examples, 51020 words/s, in_qsiz

2020-05-04 19:15:07,602 : INFO : EPOCH 44 - PROGRESS: at 57.46% examples, 44503 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:15:08,626 : INFO : EPOCH 44 - PROGRESS: at 68.08% examples, 45425 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:15:09,635 : INFO : EPOCH 44 - PROGRESS: at 79.12% examples, 46450 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:15:10,640 : INFO : EPOCH 44 - PROGRESS: at 89.78% examples, 47299 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:15:11,369 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:15:11,374 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:15:11,376 : INFO : EPOCH - 44 : training on 4871166 raw words (471484 effective words) took 9.9s, 47847 effective words/s
2020-05-04 19:15:12,404 : INFO : EPOCH 45 - PROGRESS: at 12.01% examples, 53952 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:15:13,406 : INFO : EPOCH 45 - PROGRESS: at 22.91% examples, 54563 words/s, in_qsize 4, out_qsize 0
2020

2020-05-04 19:16:07,529 : INFO : EPOCH 50 - PROGRESS: at 92.88% examples, 49026 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:16:07,980 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:16:07,984 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:16:07,986 : INFO : EPOCH - 50 : training on 4871166 raw words (471430 effective words) took 9.6s, 49295 effective words/s
2020-05-04 19:16:07,987 : INFO : training on a 243558300 raw words (23578896 effective words) took 468.9s, 50287 effective words/s


In [53]:
result=[]
for query in queries:
    rank=model.docvecs.most_similar(positive=[model.infer_vector(query,alpha=0.01,steps=1000)],topn=2914)
    result.append(rank)

2020-05-04 19:29:49,785 : INFO : precomputing L2-norms of doc weight vectors


In [57]:
file=open(output_file,'w')
for i in range(11,51):
    for j in range(len(result[i-1])):
        line="AILA_Q"+str(i)+" "+"Q0 "+str(result[i-1][j][0])+" "+str(j+1)+" "+str(result[i-1][j][1])
        file.write(line)
        file.write("\n")
file.close()

## Run2

In [48]:
Documents_train=Documents+queries[0:10]
Documents_train = [TaggedDocument(doc, [label]) for label, doc in zip(labels,Documents_train)]

In [56]:
saved_path = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec100d_run2.bin" #output model
pretrained_emb = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/law2vec embeddings/Law2Vec.100d.txt" #pretrained word embeddings
output_file="/home/spaggy/Documents/NLP Project 2020/AILA-dataset/Evaluation_061/evalutaion_file_law2vec100d_run2.txt" #Predicted output

In [50]:
#doc2vec parameters
vector_size = 100
window_size = 20
min_count = 1
sampling_threshold = 1e-6
negative_size = 5
train_epoch =50
dm = 1 #0 = dbow; 1 = dmpv
worker_count = 2 #number of parallel processes

In [51]:
#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#train doc2vec model
#docs = g.doc2vec.TaggedLineDocument(train_corpus)
#model = g.Doc2Vec(Documents_train, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0,pretrained_emb=pretrained_emb, dm=1, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)
model = g.Doc2Vec(Documents_train, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)


2020-05-04 19:21:48,386 : INFO : using concatenative 4100-dimensional layer1
2020-05-04 19:21:48,390 : INFO : collecting all words and their counts
2020-05-04 19:21:48,392 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-04 19:21:49,136 : INFO : collected 56484 word types and 2924 unique tags from a corpus of 2924 examples and 4860647 words
2020-05-04 19:21:49,137 : INFO : Loading a fresh vocabulary
2020-05-04 19:21:49,239 : INFO : effective_min_count=1 retains 56484 unique words (100% of original 56484, drops 0)
2020-05-04 19:21:49,241 : INFO : effective_min_count=1 leaves 4860647 word corpus (100% of original 4860647, drops 0)
2020-05-04 19:21:49,371 : INFO : deleting the raw counts dictionary of 56484 items
2020-05-04 19:21:49,375 : INFO : sample=1e-06 downsamples 10663 most-common words
2020-05-04 19:21:49,377 : INFO : downsampling leaves estimated 467711 word corpus (9.6% of prior 4860647)
2020-05-04 19:21:49,519 : INFO : estimated required m

2020-05-04 19:22:41,247 : INFO : EPOCH - 5 : training on 4860647 raw words (471185 effective words) took 9.4s, 50140 effective words/s
2020-05-04 19:22:42,269 : INFO : EPOCH 6 - PROGRESS: at 11.42% examples, 51229 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:22:43,298 : INFO : EPOCH 6 - PROGRESS: at 22.16% examples, 52020 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:22:44,323 : INFO : EPOCH 6 - PROGRESS: at 33.28% examples, 51467 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:22:45,331 : INFO : EPOCH 6 - PROGRESS: at 45.18% examples, 51242 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:22:46,343 : INFO : EPOCH 6 - PROGRESS: at 55.75% examples, 51238 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:22:47,343 : INFO : EPOCH 6 - PROGRESS: at 67.27% examples, 51577 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:22:48,358 : INFO : EPOCH 6 - PROGRESS: at 76.74% examples, 50613 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:22:49,360 : INFO : EPOCH 6 - PROGRESS: at 86.90% examples, 50649 wo

2020-05-04 19:23:40,915 : INFO : EPOCH 12 - PROGRESS: at 21.92% examples, 52765 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:23:41,921 : INFO : EPOCH 12 - PROGRESS: at 33.28% examples, 52685 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:23:42,934 : INFO : EPOCH 12 - PROGRESS: at 45.73% examples, 52686 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:23:43,950 : INFO : EPOCH 12 - PROGRESS: at 57.18% examples, 52837 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:23:44,962 : INFO : EPOCH 12 - PROGRESS: at 68.47% examples, 52936 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:23:45,969 : INFO : EPOCH 12 - PROGRESS: at 78.52% examples, 52323 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:23:46,987 : INFO : EPOCH 12 - PROGRESS: at 88.41% examples, 52030 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:23:47,987 : INFO : EPOCH 12 - PROGRESS: at 99.59% examples, 51816 words/s, in_qsize 1, out_qsize 1
2020-05-04 19:23:47,990 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-

2020-05-04 19:24:39,904 : INFO : EPOCH 18 - PROGRESS: at 54.07% examples, 50318 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:24:40,929 : INFO : EPOCH 18 - PROGRESS: at 64.50% examples, 49481 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:24:41,938 : INFO : EPOCH 18 - PROGRESS: at 74.83% examples, 49573 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:24:42,963 : INFO : EPOCH 18 - PROGRESS: at 82.90% examples, 48300 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:24:43,981 : INFO : EPOCH 18 - PROGRESS: at 93.60% examples, 48614 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:24:44,520 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:24:44,529 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:24:44,531 : INFO : EPOCH - 18 : training on 4860647 raw words (471163 effective words) took 9.7s, 48748 effective words/s
2020-05-04 19:24:45,544 : INFO : EPOCH 19 - PROGRESS: at 11.94% examples, 53866 words/s, in_qsize 4, out_qsize 0
2020

2020-05-04 19:25:39,165 : INFO : EPOCH 24 - PROGRESS: at 73.50% examples, 48697 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:25:40,191 : INFO : EPOCH 24 - PROGRESS: at 84.10% examples, 49098 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:25:41,201 : INFO : EPOCH 24 - PROGRESS: at 95.21% examples, 49448 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:25:41,589 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:25:41,591 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:25:41,593 : INFO : EPOCH - 24 : training on 4860647 raw words (470595 effective words) took 9.5s, 49527 effective words/s
2020-05-04 19:25:42,600 : INFO : EPOCH 25 - PROGRESS: at 11.76% examples, 53286 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:25:43,613 : INFO : EPOCH 25 - PROGRESS: at 22.71% examples, 53769 words/s, in_qsize 3, out_qsize 1
2020-05-04 19:25:44,634 : INFO : EPOCH 25 - PROGRESS: at 34.13% examples, 53514 words/s, in_qsize 4, out_qsize 0
2020

2020-05-04 19:26:37,293 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:26:37,303 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:26:37,305 : INFO : EPOCH - 30 : training on 4860647 raw words (471445 effective words) took 9.8s, 48156 effective words/s
2020-05-04 19:26:38,321 : INFO : EPOCH 31 - PROGRESS: at 10.16% examples, 46389 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:26:39,337 : INFO : EPOCH 31 - PROGRESS: at 20.96% examples, 49711 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:26:40,338 : INFO : EPOCH 31 - PROGRESS: at 33.07% examples, 51599 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:26:41,347 : INFO : EPOCH 31 - PROGRESS: at 45.73% examples, 52448 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:26:42,350 : INFO : EPOCH 31 - PROGRESS: at 57.39% examples, 52954 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:26:43,358 : INFO : EPOCH 31 - PROGRESS: at 68.47% examples, 52989 words/s, in_qsize 4, out_qsize 0
2020

2020-05-04 19:27:36,384 : INFO : EPOCH 37 - PROGRESS: at 69.46% examples, 54114 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:27:37,397 : INFO : EPOCH 37 - PROGRESS: at 81.26% examples, 54048 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:27:38,419 : INFO : EPOCH 37 - PROGRESS: at 92.41% examples, 54175 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:27:39,038 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:27:39,040 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:27:39,042 : INFO : EPOCH - 37 : training on 4860647 raw words (470349 effective words) took 8.7s, 54134 effective words/s
2020-05-04 19:27:40,047 : INFO : EPOCH 38 - PROGRESS: at 12.18% examples, 55200 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:27:41,051 : INFO : EPOCH 38 - PROGRESS: at 23.22% examples, 55320 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:27:42,069 : INFO : EPOCH 38 - PROGRESS: at 35.05% examples, 54871 words/s, in_qsize 4, out_qsize 0
2020

2020-05-04 19:28:33,759 : INFO : EPOCH 44 - PROGRESS: at 11.59% examples, 52506 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:28:34,762 : INFO : EPOCH 44 - PROGRESS: at 21.37% examples, 51254 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:28:35,780 : INFO : EPOCH 44 - PROGRESS: at 32.59% examples, 50876 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:28:36,798 : INFO : EPOCH 44 - PROGRESS: at 44.63% examples, 51061 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:28:37,814 : INFO : EPOCH 44 - PROGRESS: at 55.75% examples, 51589 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:28:38,830 : INFO : EPOCH 44 - PROGRESS: at 67.41% examples, 51872 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:28:39,844 : INFO : EPOCH 44 - PROGRESS: at 78.52% examples, 52115 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:28:40,846 : INFO : EPOCH 44 - PROGRESS: at 88.61% examples, 52111 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:28:41,798 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-

2020-05-04 19:29:34,098 : INFO : EPOCH 50 - PROGRESS: at 69.46% examples, 54327 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:29:35,111 : INFO : EPOCH 50 - PROGRESS: at 81.26% examples, 54196 words/s, in_qsize 3, out_qsize 0
2020-05-04 19:29:36,123 : INFO : EPOCH 50 - PROGRESS: at 91.52% examples, 53833 words/s, in_qsize 4, out_qsize 0
2020-05-04 19:29:36,816 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 19:29:36,818 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 19:29:36,820 : INFO : EPOCH - 50 : training on 4860647 raw words (470963 effective words) took 8.7s, 53841 effective words/s
2020-05-04 19:29:36,821 : INFO : training on a 243032350 raw words (23532762 effective words) took 466.8s, 50411 effective words/s


In [52]:
model.save(saved_path)

2020-05-04 19:29:39,653 : INFO : saving Doc2Vec object under /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec100d_run2.bin, separately None
2020-05-04 19:29:39,658 : INFO : storing np array 'syn1neg' to /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec100d_run2.bin.trainables.syn1neg.npy
2020-05-04 19:29:47,115 : INFO : saved /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec100d_run2.bin


In [53]:
result=[]
for query in queries:
    rank=model.docvecs.most_similar(positive=[model.infer_vector(query,alpha=0.01,steps=1000)],topn=2914)
    result.append(rank)

2020-05-04 19:29:49,785 : INFO : precomputing L2-norms of doc weight vectors


In [54]:
result[41]

[('C150', 0.4108192026615143),
 ('C1826', 0.3966875672340393),
 ('C659', 0.38744789361953735),
 ('C2669', 0.38296812772750854),
 ('C1032', 0.36280447244644165),
 ('C377', 0.3623620867729187),
 ('C188', 0.3544568121433258),
 ('C1114', 0.3489874601364136),
 ('C425', 0.33567196130752563),
 ('C1382', 0.3355760872364044),
 ('C427', 0.3331306576728821),
 ('C877', 0.33282727003097534),
 ('C1152', 0.33245378732681274),
 ('C2192', 0.32437029480934143),
 ('C1338', 0.3242129981517792),
 ('C308', 0.3188892602920532),
 ('C372', 0.3185555636882782),
 ('C284', 0.30962151288986206),
 ('C636', 0.305703341960907),
 ('C1793', 0.30276352167129517),
 ('C2818', 0.3014283776283264),
 ('C1643', 0.3005639612674713),
 ('C400', 0.2965722680091858),
 ('C1006', 0.2957453727722168),
 ('C2162', 0.2945459187030792),
 ('C843', 0.293819397687912),
 ('C2666', 0.2938087582588196),
 ('C914', 0.29137876629829407),
 ('C677', 0.2907346487045288),
 ('C2747', 0.28997963666915894),
 ('C2149', 0.28700241446495056),
 ('C33', 0.28

In [57]:
file=open(output_file,'w')
for i in range(11,51):
    for j in range(len(result[i-1])):
        line="AILA_Q"+str(i)+" "+"Q0 "+str(result[i-1][j][0])+" "+str(j+1)+" "+str(result[i-1][j][1])
        file.write(line)
        file.write("\n")
file.close()

In [58]:
del model

## 3. Doc2Vec using Law2Vec200d

## Run1

In [5]:
Documents_train=Documents+queries
Documents_train = [TaggedDocument(doc, [label]) for label, doc in zip(labels,Documents_train)]

In [6]:
saved_path = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run1.bin" #output model
pretrained_emb = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/law2vec embeddings/Law2Vec.200d.txt" #pretrained word embeddings
output_file="/home/spaggy/Documents/NLP Project 2020/AILA-dataset/Evaluation_061/evalutaion_file_law2vec200d_run1.txt" #Predicted output

In [7]:
#doc2vec parameters
vector_size = 200
window_size = 20
min_count = 1
sampling_threshold = 1e-6
negative_size = 5
train_epoch =50
dm = 1 #0 = dbow; 1 = dmpv
worker_count = 4 #number of parallel processes


In [8]:
#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#train doc2vec model
#docs = g.doc2vec.TaggedLineDocument(train_corpus)
#model = g.Doc2Vec(Documents, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)
model = g.Doc2Vec(Documents_train, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0,pretrained_emb=pretrained_emb, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)


/home/spaggy/.local/lib/python2.7/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/spaggy/.local/lib/python2.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2020-05-04 23:00:09,296 : INFO : using concatenative 8200-dimensional layer1
2020-05-04 23:00:09,297 : INFO : collecting all words and their counts
2020-05-04 23:00:09,299 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-04 23:00:10,380 : INFO : collected 56521 word types and 2964 unique tags from a corpus of 2964 examples and 4871166 words
2020-05-04 23:00:10,382 : INFO : Loading a fresh voc

2020-05-04 23:01:04,329 : INFO : EPOCH 3 - PROGRESS: at 22.64% examples, 20703 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:01:05,363 : INFO : EPOCH 3 - PROGRESS: at 27.73% examples, 20855 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:01:06,427 : INFO : EPOCH 3 - PROGRESS: at 32.22% examples, 20797 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:01:07,442 : INFO : EPOCH 3 - PROGRESS: at 37.01% examples, 20990 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:01:08,472 : INFO : EPOCH 3 - PROGRESS: at 41.70% examples, 20836 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:01:09,588 : INFO : EPOCH 3 - PROGRESS: at 46.22% examples, 20545 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:01:10,603 : INFO : EPOCH 3 - PROGRESS: at 49.93% examples, 20380 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:01:11,640 : INFO : EPOCH 3 - PROGRESS: at 54.01% examples, 20248 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:01:12,722 : INFO : EPOCH 3 - PROGRESS: at 58.70% examples, 20106 words/s, in_qsize 8, out_

2020-05-04 23:02:10,109 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 23:02:10,111 : INFO : EPOCH - 5 : training on 4871166 raw words (471943 effective words) took 23.8s, 19829 effective words/s
2020-05-04 23:02:11,148 : INFO : EPOCH 6 - PROGRESS: at 4.62% examples, 18458 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:02:12,176 : INFO : EPOCH 6 - PROGRESS: at 9.11% examples, 20758 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:02:13,241 : INFO : EPOCH 6 - PROGRESS: at 13.77% examples, 21052 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:02:14,246 : INFO : EPOCH 6 - PROGRESS: at 18.25% examples, 21311 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:02:15,296 : INFO : EPOCH 6 - PROGRESS: at 22.71% examples, 21108 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:02:16,315 : INFO : EPOCH 6 - PROGRESS: at 27.23% examples, 20942 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:02:17,404 : INFO : EPOCH 6 - PROGRESS: at 31.75% examples, 20709 words/s, in_qsize 7, out

2020-05-04 23:03:16,444 : INFO : EPOCH 8 - PROGRESS: at 86.74% examples, 20281 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:03:17,487 : INFO : EPOCH 8 - PROGRESS: at 90.92% examples, 20190 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:03:18,509 : INFO : EPOCH 8 - PROGRESS: at 94.74% examples, 20059 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:03:19,324 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-04 23:03:19,344 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-04 23:03:19,350 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 23:03:19,353 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 23:03:19,359 : INFO : EPOCH - 8 : training on 4871166 raw words (470796 effective words) took 23.5s, 20066 effective words/s
2020-05-04 23:03:20,419 : INFO : EPOCH 9 - PROGRESS: at 4.28% examples, 16672 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:03:21,445 : INFO : EPOCH 9 - PROGRESS: 

2020-05-04 23:04:19,682 : INFO : EPOCH 11 - PROGRESS: at 62.65% examples, 20326 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:04:20,697 : INFO : EPOCH 11 - PROGRESS: at 66.80% examples, 20366 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:04:21,704 : INFO : EPOCH 11 - PROGRESS: at 70.75% examples, 20360 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:04:22,713 : INFO : EPOCH 11 - PROGRESS: at 75.71% examples, 20503 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:04:23,746 : INFO : EPOCH 11 - PROGRESS: at 80.16% examples, 20512 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:04:24,741 : INFO : EPOCH 11 - PROGRESS: at 84.14% examples, 20618 words/s, in_qsize 8, out_qsize 1
2020-05-04 23:04:25,752 : INFO : EPOCH 11 - PROGRESS: at 88.33% examples, 20695 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:04:26,764 : INFO : EPOCH 11 - PROGRESS: at 93.02% examples, 20717 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:04:27,727 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-

...

2020-05-04 23:13:48,330 : INFO : EPOCH - 37 : training on 4871166 raw words (470848 effective words) took 23.2s, 20320 effective words/s
2020-05-04 23:13:49,388 : INFO : EPOCH 38 - PROGRESS: at 4.82% examples, 18536 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:13:50,410 : INFO : EPOCH 38 - PROGRESS: at 8.84% examples, 20131 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:13:51,418 : INFO : EPOCH 38 - PROGRESS: at 13.26% examples, 19834 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:13:52,435 : INFO : EPOCH 38 - PROGRESS: at 16.67% examples, 19668 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:13:53,512 : INFO : EPOCH 38 - PROGRESS: at 21.02% examples, 19154 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:13:54,568 : INFO : EPOCH 38 - PROGRESS: at 25.07% examples, 19193 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:13:55,580 : INFO : EPOCH 38 - PROGRESS: at 29.55% examples, 19347 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:13:56,593 : INFO : EPOCH 38 - PROGRESS: at 33.37% examples, 

2020-05-04 23:14:56,325 : INFO : EPOCH 40 - PROGRESS: at 88.43% examples, 20391 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:14:57,340 : INFO : EPOCH 40 - PROGRESS: at 92.88% examples, 20386 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:14:58,390 : INFO : EPOCH 40 - PROGRESS: at 97.06% examples, 20289 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:14:58,574 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-04 23:14:58,625 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-04 23:14:58,661 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 23:14:58,666 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 23:14:58,668 : INFO : EPOCH - 40 : training on 4871166 raw words (472365 effective words) took 23.3s, 20307 effective words/s
2020-05-04 23:14:59,677 : INFO : EPOCH 41 - PROGRESS: at 4.52% examples, 18167 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:15:00,691 : INFO : EPOCH 41 - PROG

2020-05-04 23:15:59,836 : INFO : EPOCH 43 - PROGRESS: at 62.42% examples, 20405 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:16:00,867 : INFO : EPOCH 43 - PROGRESS: at 65.79% examples, 20190 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:16:01,883 : INFO : EPOCH 43 - PROGRESS: at 69.84% examples, 20194 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:16:02,884 : INFO : EPOCH 43 - PROGRESS: at 74.09% examples, 20190 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:16:03,924 : INFO : EPOCH 43 - PROGRESS: at 78.21% examples, 20176 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:16:04,982 : INFO : EPOCH 43 - PROGRESS: at 82.42% examples, 20174 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:16:06,011 : INFO : EPOCH 43 - PROGRESS: at 86.57% examples, 20267 words/s, in_qsize 8, out_qsize 1
2020-05-04 23:16:07,080 : INFO : EPOCH 43 - PROGRESS: at 91.33% examples, 20217 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:16:08,111 : INFO : EPOCH 43 - PROGRESS: at 95.95% examples, 20281 words/s, in_qsiz

2020-05-04 23:17:03,141 : INFO : EPOCH 46 - PROGRESS: at 36.84% examples, 21411 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:17:04,195 : INFO : EPOCH 46 - PROGRESS: at 41.70% examples, 21214 words/s, in_qsize 7, out_qsize 1
2020-05-04 23:17:05,222 : INFO : EPOCH 46 - PROGRESS: at 46.63% examples, 21229 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:17:06,226 : INFO : EPOCH 46 - PROGRESS: at 50.47% examples, 21127 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:17:07,253 : INFO : EPOCH 46 - PROGRESS: at 55.50% examples, 21288 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:17:08,256 : INFO : EPOCH 46 - PROGRESS: at 60.39% examples, 21239 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:17:09,289 : INFO : EPOCH 46 - PROGRESS: at 64.81% examples, 21337 words/s, in_qsize 7, out_qsize 0
2020-05-04 23:17:10,329 : INFO : EPOCH 46 - PROGRESS: at 68.96% examples, 21321 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:17:11,383 : INFO : EPOCH 46 - PROGRESS: at 74.22% examples, 21400 words/s, in_qsiz

2020-05-04 23:18:05,850 : INFO : EPOCH 49 - PROGRESS: at 12.89% examples, 19459 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:18:06,853 : INFO : EPOCH 49 - PROGRESS: at 16.46% examples, 18935 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:18:07,865 : INFO : EPOCH 49 - PROGRESS: at 20.45% examples, 19474 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:18:08,882 : INFO : EPOCH 49 - PROGRESS: at 24.60% examples, 19327 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:18:09,928 : INFO : EPOCH 49 - PROGRESS: at 28.85% examples, 19126 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:18:10,951 : INFO : EPOCH 49 - PROGRESS: at 32.62% examples, 19051 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:18:11,960 : INFO : EPOCH 49 - PROGRESS: at 37.28% examples, 19277 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:18:12,980 : INFO : EPOCH 49 - PROGRESS: at 42.04% examples, 19358 words/s, in_qsize 8, out_qsize 0
2020-05-04 23:18:14,000 : INFO : EPOCH 49 - PROGRESS: at 45.88% examples, 19213 words/s, in_qsiz

In [9]:
model.save(saved_path)

2020-05-04 23:18:58,210 : INFO : saving Doc2Vec object under /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run1.bin, separately None
2020-05-04 23:18:58,212 : INFO : storing np array 'vectors' to /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run1.bin.wv.vectors.npy
2020-05-04 23:18:59,895 : INFO : storing np array 'syn1neg' to /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run1.bin.trainables.syn1neg.npy
2020-05-04 23:19:21,665 : INFO : saved /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run1.bin


In [10]:
result=[]
for query in queries:
    rank=model.docvecs.most_similar(positive=[model.infer_vector(query,alpha=0.01,steps=1000)],topn=2914)
    result.append(rank)

2020-05-04 23:19:22,991 : INFO : precomputing L2-norms of doc weight vectors


In [13]:
result[13]

[('AILA14', 0.7177094221115112),
 ('C46', 0.5060343742370605),
 ('C2904', 0.38802874088287354),
 ('C2406', 0.3488018214702606),
 ('C2714', 0.33944278955459595),
 ('C333', 0.3384104371070862),
 ('C947', 0.3354787826538086),
 ('C2537', 0.33117473125457764),
 ('C1984', 0.3294142186641693),
 ('C1935', 0.32564982771873474),
 ('C528', 0.32470571994781494),
 ('C832', 0.3212587833404541),
 ('C2493', 0.31850847601890564),
 ('C1394', 0.318426251411438),
 ('C1964', 0.31772056221961975),
 ('C1100', 0.3164318799972534),
 ('C2155', 0.30607593059539795),
 ('C548', 0.294411838054657),
 ('C2433', 0.2926715612411499),
 ('C983', 0.29147762060165405),
 ('C1143', 0.29079362750053406),
 ('C586', 0.2904629111289978),
 ('C1145', 0.28982651233673096),
 ('C2606', 0.2895330786705017),
 ('C2859', 0.2876841127872467),
 ('C1777', 0.28526371717453003),
 ('C534', 0.28430649638175964),
 ('C718', 0.2832205295562744),
 ('C1589', 0.2824072241783142),
 ('C2089', 0.28188860416412354),
 ('C1347', 0.279888391494751),
 ('C169

In [12]:
file=open(output_file,'w')
for i in range(11,51):
    for j in range(len(result[i-1])):
        line="AILA_Q"+str(i)+" "+"Q0 "+str(result[i-1][j][0])+" "+str(j+1)+" "+str(result[i-1][j][1])
        file.write(line)
        file.write("\n")
file.close()

In [15]:
del model

## Run2

In [5]:
Documents_train=Documents+queries[0:10]
Documents_train = [TaggedDocument(doc, [label]) for label, doc in zip(labels,Documents_train)]

In [11]:
saved_path = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run2.bin" #output model
pretrained_emb = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/law2vec embeddings/Law2Vec.200d.txt" #pretrained word embeddings
output_file="/home/spaggy/Documents/NLP Project 2020/AILA-dataset/Evaluation_061/evalutaion_file_law2vec200d_run2.txt" #Predicted output

In [9]:
#doc2vec parameters
vector_size = 200
window_size = 20
min_count = 1
sampling_threshold = 1e-6
negative_size = 5
train_epoch =50
dm = 1 #0 = dbow; 1 = dmpv
worker_count = 4 #number of parallel processes


In [10]:
#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#train doc2vec model
#docs = g.doc2vec.TaggedLineDocument(train_corpus)
#model = g.Doc2Vec(Documents, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)
model = g.Doc2Vec(Documents_train, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0,pretrained_emb=pretrained_emb, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)


2020-05-04 22:14:01,808 : INFO : using concatenative 8200-dimensional layer1
2020-05-04 22:14:01,812 : INFO : collecting all words and their counts
2020-05-04 22:14:01,813 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-04 22:14:02,604 : INFO : collected 56484 word types and 2924 unique tags from a corpus of 2924 examples and 4860647 words
2020-05-04 22:14:02,605 : INFO : Loading a fresh vocabulary
2020-05-04 22:14:02,915 : INFO : effective_min_count=1 retains 56484 unique words (100% of original 56484, drops 0)
2020-05-04 22:14:02,917 : INFO : effective_min_count=1 leaves 4860647 word corpus (100% of original 4860647, drops 0)
2020-05-04 22:14:03,048 : INFO : deleting the raw counts dictionary of 56484 items
2020-05-04 22:14:03,052 : INFO : sample=1e-06 downsamples 10663 most-common words
2020-05-04 22:14:03,053 : INFO : downsampling leaves estimated 467711 word corpus (9.6% of prior 4860647)
2020-05-04 22:14:03,210 : INFO : estimated required m

2020-05-04 22:15:00,670 : INFO : EPOCH 3 - PROGRESS: at 88.61% examples, 22709 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:15:01,672 : INFO : EPOCH 3 - PROGRESS: at 94.15% examples, 22837 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:15:02,620 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-04 22:15:02,637 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-04 22:15:02,647 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 22:15:02,651 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 22:15:02,653 : INFO : EPOCH - 3 : training on 4860647 raw words (471422 effective words) took 20.6s, 22927 effective words/s
2020-05-04 22:15:03,712 : INFO : EPOCH 4 - PROGRESS: at 5.57% examples, 21558 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:15:04,740 : INFO : EPOCH 4 - PROGRESS: at 10.36% examples, 22950 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:15:05,763 : INFO : EPOCH 4 - PROGRESS: 

2020-05-04 22:16:00,713 : INFO : EPOCH 7 - PROGRESS: at 11.59% examples, 25939 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:16:01,731 : INFO : EPOCH 7 - PROGRESS: at 16.89% examples, 26632 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:16:02,747 : INFO : EPOCH 7 - PROGRESS: at 22.95% examples, 26921 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:16:03,783 : INFO : EPOCH 7 - PROGRESS: at 28.69% examples, 26403 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:16:04,798 : INFO : EPOCH 7 - PROGRESS: at 33.48% examples, 26012 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:16:05,859 : INFO : EPOCH 7 - PROGRESS: at 39.71% examples, 25965 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:16:06,881 : INFO : EPOCH 7 - PROGRESS: at 46.14% examples, 26159 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:16:07,906 : INFO : EPOCH 7 - PROGRESS: at 51.40% examples, 25977 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:16:08,917 : INFO : EPOCH 7 - PROGRESS: at 56.91% examples, 25918 words/s, in_qsize 7, out_

2020-05-04 22:17:03,703 : INFO : EPOCH 10 - PROGRESS: at 36.97% examples, 24064 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:17:04,735 : INFO : EPOCH 10 - PROGRESS: at 42.85% examples, 24124 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:17:05,745 : INFO : EPOCH 10 - PROGRESS: at 48.08% examples, 24083 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:17:06,818 : INFO : EPOCH 10 - PROGRESS: at 52.60% examples, 24018 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:17:07,887 : INFO : EPOCH 10 - PROGRESS: at 59.68% examples, 24210 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:17:08,901 : INFO : EPOCH 10 - PROGRESS: at 65.18% examples, 24442 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:17:09,922 : INFO : EPOCH 10 - PROGRESS: at 69.46% examples, 24318 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:17:10,940 : INFO : EPOCH 10 - PROGRESS: at 75.10% examples, 24334 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:17:11,955 : INFO : EPOCH 10 - PROGRESS: at 80.85% examples, 24521 words/s, in_qsiz

2020-05-04 22:18:06,144 : INFO : EPOCH 13 - PROGRESS: at 59.68% examples, 24431 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:18:07,147 : INFO : EPOCH 13 - PROGRESS: at 64.74% examples, 24468 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:18:08,174 : INFO : EPOCH 13 - PROGRESS: at 69.66% examples, 24548 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:18:09,216 : INFO : EPOCH 13 - PROGRESS: at 75.89% examples, 24690 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:18:10,252 : INFO : EPOCH 13 - PROGRESS: at 81.57% examples, 24795 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:18:11,260 : INFO : EPOCH 13 - PROGRESS: at 86.73% examples, 24980 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:18:12,278 : INFO : EPOCH 13 - PROGRESS: at 92.07% examples, 24967 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:18:13,299 : INFO : EPOCH 13 - PROGRESS: at 96.99% examples, 24837 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:18:13,725 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-

...

2020-05-04 22:26:35,643 : INFO : EPOCH 37 - PROGRESS: at 44.94% examples, 25399 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:26:36,660 : INFO : EPOCH 37 - PROGRESS: at 50.27% examples, 25466 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:26:37,714 : INFO : EPOCH 37 - PROGRESS: at 55.75% examples, 25415 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:26:38,723 : INFO : EPOCH 37 - PROGRESS: at 61.97% examples, 25484 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:26:39,820 : INFO : EPOCH 37 - PROGRESS: at 66.79% examples, 25239 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:26:40,887 : INFO : EPOCH 37 - PROGRESS: at 72.37% examples, 25257 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:26:41,918 : INFO : EPOCH 37 - PROGRESS: at 78.08% examples, 25263 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:26:42,979 : INFO : EPOCH 37 - PROGRESS: at 83.00% examples, 25150 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:26:44,023 : INFO : EPOCH 37 - PROGRESS: at 88.03% examples, 25242 words/s, in_qsiz

2020-05-04 22:27:39,233 : INFO : EPOCH 40 - PROGRESS: at 87.65% examples, 25408 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:27:40,238 : INFO : EPOCH 40 - PROGRESS: at 92.78% examples, 25283 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:27:41,243 : INFO : EPOCH 40 - PROGRESS: at 98.36% examples, 25372 words/s, in_qsize 6, out_qsize 0
2020-05-04 22:27:41,353 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-04 22:27:41,392 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-04 22:27:41,405 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 22:27:41,419 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 22:27:41,421 : INFO : EPOCH - 40 : training on 4860647 raw words (471081 effective words) took 18.6s, 25388 effective words/s
2020-05-04 22:27:42,432 : INFO : EPOCH 41 - PROGRESS: at 6.43% examples, 26011 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:27:43,434 : INFO : EPOCH 41 - PROG

2020-05-04 22:28:37,787 : INFO : EPOCH 44 - PROGRESS: at 11.01% examples, 24856 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:28:38,811 : INFO : EPOCH 44 - PROGRESS: at 16.21% examples, 24480 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:28:39,842 : INFO : EPOCH 44 - PROGRESS: at 21.27% examples, 25080 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:28:40,862 : INFO : EPOCH 44 - PROGRESS: at 27.87% examples, 25770 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:28:41,866 : INFO : EPOCH 44 - PROGRESS: at 33.48% examples, 26162 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:28:42,883 : INFO : EPOCH 44 - PROGRESS: at 40.18% examples, 26528 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:28:43,904 : INFO : EPOCH 44 - PROGRESS: at 46.85% examples, 26693 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:28:44,929 : INFO : EPOCH 44 - PROGRESS: at 52.43% examples, 27055 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:28:45,961 : INFO : EPOCH 44 - PROGRESS: at 59.88% examples, 27218 words/s, in_qsiz

2020-05-04 22:29:40,398 : INFO : EPOCH 47 - PROGRESS: at 55.23% examples, 25141 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:29:41,415 : INFO : EPOCH 47 - PROGRESS: at 60.53% examples, 24898 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:29:42,421 : INFO : EPOCH 47 - PROGRESS: at 65.22% examples, 24810 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:29:43,498 : INFO : EPOCH 47 - PROGRESS: at 70.11% examples, 24710 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:29:44,529 : INFO : EPOCH 47 - PROGRESS: at 76.03% examples, 24822 words/s, in_qsize 7, out_qsize 0
2020-05-04 22:29:45,540 : INFO : EPOCH 47 - PROGRESS: at 82.08% examples, 25058 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:29:46,603 : INFO : EPOCH 47 - PROGRESS: at 87.55% examples, 25182 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:29:47,642 : INFO : EPOCH 47 - PROGRESS: at 93.37% examples, 25334 words/s, in_qsize 8, out_qsize 0
2020-05-04 22:29:48,668 : INFO : EPOCH 47 - PROGRESS: at 99.01% examples, 25344 words/s, in_qsiz

2020-05-04 22:30:42,680 : INFO : EPOCH 50 - PROGRESS: at 96.72% examples, 26220 words/s, in_qsize 7, out_qsize 1
2020-05-04 22:30:43,105 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-04 22:30:43,122 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-04 22:30:43,145 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 22:30:43,154 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 22:30:43,155 : INFO : EPOCH - 50 : training on 4860647 raw words (470753 effective words) took 17.9s, 26271 effective words/s
2020-05-04 22:30:43,157 : INFO : training on a 243032350 raw words (23538864 effective words) took 999.4s, 23554 effective words/s


In [12]:
model.save(saved_path)

2020-05-04 22:31:50,096 : INFO : saving Doc2Vec object under /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run2.bin, separately None
2020-05-04 22:31:50,101 : INFO : storing np array 'vectors' to /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run2.bin.wv.vectors.npy
2020-05-04 22:31:50,248 : INFO : storing np array 'syn1neg' to /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run2.bin.trainables.syn1neg.npy
2020-05-04 22:32:09,766 : INFO : saved /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_law2vec200d_run2.bin


In [13]:
result=[]
for query in queries:
    rank=model.docvecs.most_similar(positive=[model.infer_vector(query,alpha=0.01,steps=1000)],topn=2914)
    result.append(rank)

2020-05-04 22:32:12,905 : INFO : precomputing L2-norms of doc weight vectors


In [14]:
result[10]

[('C1354', 0.47439730167388916),
 ('C1047', 0.4418579339981079),
 ('C730', 0.4297432601451874),
 ('C2787', 0.42104101181030273),
 ('C2730', 0.39596396684646606),
 ('C622', 0.3862205743789673),
 ('C1547', 0.3831115663051605),
 ('C1123', 0.3781305253505707),
 ('C2389', 0.37266814708709717),
 ('C16', 0.3687087297439575),
 ('C737', 0.363225519657135),
 ('C452', 0.36299967765808105),
 ('C2351', 0.3625492453575134),
 ('C272', 0.3607226312160492),
 ('C2156', 0.3568253517150879),
 ('C2643', 0.35568124055862427),
 ('C598', 0.35347139835357666),
 ('C686', 0.3523370325565338),
 ('C1398', 0.3507382869720459),
 ('C1731', 0.34799981117248535),
 ('C602', 0.34368669986724854),
 ('C1794', 0.34036359190940857),
 ('C2506', 0.33636945486068726),
 ('C217', 0.3362095057964325),
 ('C1042', 0.33613142371177673),
 ('C1653', 0.33597689867019653),
 ('C136', 0.32836389541625977),
 ('C2433', 0.32809215784072876),
 ('C2897', 0.32525938749313354),
 ('C2246', 0.3251217305660248),
 ('C1642', 0.3250456154346466),
 ('C7

In [15]:
file=open(output_file,'w')
for i in range(11,51):
    for j in range(len(result[i-1])):
        line="AILA_Q"+str(i)+" "+"Q0 "+str(result[i-1][j][0])+" "+str(j+1)+" "+str(result[i-1][j][1])
        file.write(line)
        file.write("\n")
file.close()

In [95]:
del model

## 4. Doc2Vec DMPV

In [16]:
saved_path = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_dmpv.bin" #output model
#pretrained_emb = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/law2vec embeddings/Law2Vec.200d.txt" #pretrained word embeddings
output_file="/home/spaggy/Documents/NLP Project 2020/AILA-dataset/Evaluation_061/evalutaion_file_dmpv.txt" #Predicted output

In [17]:
#doc2vec parameters
vector_size = 100
window_size = 20
min_count = 1
sampling_threshold = 1e-6
negative_size = 5
train_epoch =50
dm = 1 #0 = dbow; 1 = dmpv
worker_count = 2 #number of parallel processes


In [18]:
#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#train doc2vec model
#docs = g.doc2vec.TaggedLineDocument(train_corpus)
#model = g.Doc2Vec(Documents, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)
model = g.Doc2Vec(Documents_train, size=vector_size, window=window_size, min_count=min_count, workers=worker_count,dm=dm, dbow_words=1, dm_concat=1, iter=train_epoch)


2020-05-04 20:57:40,125 : INFO : using concatenative 4100-dimensional layer1
2020-05-04 20:57:40,127 : INFO : collecting all words and their counts
2020-05-04 20:57:40,128 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-04 20:57:41,075 : INFO : collected 56521 word types and 2964 unique tags from a corpus of 2964 examples and 4871166 words
2020-05-04 20:57:41,076 : INFO : Loading a fresh vocabulary
2020-05-04 20:57:41,191 : INFO : effective_min_count=1 retains 56521 unique words (100% of original 56521, drops 0)
2020-05-04 20:57:41,192 : INFO : effective_min_count=1 leaves 4871166 word corpus (100% of original 4871166, drops 0)
2020-05-04 20:57:41,358 : INFO : deleting the raw counts dictionary of 56521 items
2020-05-04 20:57:41,361 : INFO : sample=0.001 downsamples 37 most-common words
2020-05-04 20:57:41,363 : INFO : downsampling leaves estimated 4567386 word corpus (93.8% of prior 4871166)
2020-05-04 20:57:41,521 : INFO : estimated required me

2020-05-04 20:58:51,905 : INFO : EPOCH 1 - PROGRESS: at 82.46% examples, 53904 words/s, in_qsize 4, out_qsize 0
2020-05-04 20:58:52,964 : INFO : EPOCH 1 - PROGRESS: at 83.74% examples, 53932 words/s, in_qsize 4, out_qsize 0
2020-05-04 20:58:54,012 : INFO : EPOCH 1 - PROGRESS: at 85.05% examples, 54060 words/s, in_qsize 3, out_qsize 0
2020-05-04 20:58:55,024 : INFO : EPOCH 1 - PROGRESS: at 86.30% examples, 54192 words/s, in_qsize 4, out_qsize 0
2020-05-04 20:58:56,076 : INFO : EPOCH 1 - PROGRESS: at 87.42% examples, 54270 words/s, in_qsize 4, out_qsize 0
2020-05-04 20:58:57,139 : INFO : EPOCH 1 - PROGRESS: at 88.80% examples, 54340 words/s, in_qsize 3, out_qsize 0
2020-05-04 20:58:58,196 : INFO : EPOCH 1 - PROGRESS: at 90.52% examples, 54398 words/s, in_qsize 4, out_qsize 0
2020-05-04 20:58:59,235 : INFO : EPOCH 1 - PROGRESS: at 91.63% examples, 54499 words/s, in_qsize 3, out_qsize 0
2020-05-04 20:59:00,291 : INFO : EPOCH 1 - PROGRESS: at 92.88% examples, 54531 words/s, in_qsize 4, out_

2020-05-04 21:00:09,193 : INFO : EPOCH 2 - PROGRESS: at 93.25% examples, 66036 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:00:10,258 : INFO : EPOCH 2 - PROGRESS: at 95.24% examples, 66176 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:00:11,453 : INFO : EPOCH 2 - PROGRESS: at 96.90% examples, 66210 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:00:12,162 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-04 21:00:12,210 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-04 21:00:12,212 : INFO : EPOCH - 2 : training on 4871166 raw words (4511527 effective words) took 67.9s, 66464 effective words/s
2020-05-04 21:00:13,321 : INFO : EPOCH 3 - PROGRESS: at 2.02% examples, 66489 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:00:14,334 : INFO : EPOCH 3 - PROGRESS: at 3.85% examples, 72136 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:00:15,350 : INFO : EPOCH 3 - PROGRESS: at 5.87% examples, 74294 words/s, in_qsize 4, out_qsize 0
2020-05-04 2

2020-05-04 21:01:22,764 : INFO : EPOCH 4 - PROGRESS: at 43.22% examples, 99757 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:01:23,770 : INFO : EPOCH 4 - PROGRESS: at 45.31% examples, 99900 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:01:24,850 : INFO : EPOCH 4 - PROGRESS: at 47.77% examples, 100298 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:01:25,898 : INFO : EPOCH 4 - PROGRESS: at 50.10% examples, 100998 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:01:26,931 : INFO : EPOCH 4 - PROGRESS: at 52.46% examples, 101363 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:01:28,053 : INFO : EPOCH 4 - PROGRESS: at 55.50% examples, 101815 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:01:29,146 : INFO : EPOCH 4 - PROGRESS: at 58.70% examples, 102383 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:01:30,174 : INFO : EPOCH 4 - PROGRESS: at 61.34% examples, 102982 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:01:31,261 : INFO : EPOCH 4 - PROGRESS: at 63.53% examples, 103289 words/s, in_qsize 

2020-05-04 21:02:34,699 : INFO : EPOCH 6 - PROGRESS: at 15.59% examples, 109910 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:02:35,845 : INFO : EPOCH 6 - PROGRESS: at 18.15% examples, 110697 words/s, in_qsize 3, out_qsize 1
2020-05-04 21:02:36,873 : INFO : EPOCH 6 - PROGRESS: at 20.88% examples, 111095 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:02:37,969 : INFO : EPOCH 6 - PROGRESS: at 23.72% examples, 110897 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:02:38,994 : INFO : EPOCH 6 - PROGRESS: at 26.25% examples, 110948 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:02:40,000 : INFO : EPOCH 6 - PROGRESS: at 28.91% examples, 111172 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:02:41,027 : INFO : EPOCH 6 - PROGRESS: at 31.14% examples, 110514 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:02:42,040 : INFO : EPOCH 6 - PROGRESS: at 33.23% examples, 110760 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:02:43,081 : INFO : EPOCH 6 - PROGRESS: at 36.03% examples, 110837 words/s, in_qsiz

...

2020-05-04 21:26:29,973 : INFO : EPOCH 42 - PROGRESS: at 49.53% examples, 106185 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:26:31,013 : INFO : EPOCH 42 - PROGRESS: at 51.89% examples, 106960 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:26:32,128 : INFO : EPOCH 42 - PROGRESS: at 55.36% examples, 107886 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:26:33,161 : INFO : EPOCH 42 - PROGRESS: at 58.87% examples, 108826 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:26:34,177 : INFO : EPOCH 42 - PROGRESS: at 61.67% examples, 109506 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:26:35,191 : INFO : EPOCH 42 - PROGRESS: at 64.04% examples, 110243 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:26:36,225 : INFO : EPOCH 42 - PROGRESS: at 66.60% examples, 110681 words/s, in_qsize 3, out_qsize 1
2020-05-04 21:26:37,311 : INFO : EPOCH 42 - PROGRESS: at 69.16% examples, 111219 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:26:38,331 : INFO : EPOCH 42 - PROGRESS: at 71.90% examples, 111278 words/s

2020-05-04 21:27:40,972 : INFO : EPOCH 44 - PROGRESS: at 37.45% examples, 123901 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:27:41,989 : INFO : EPOCH 44 - PROGRESS: at 40.59% examples, 124078 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:27:43,018 : INFO : EPOCH 44 - PROGRESS: at 43.52% examples, 124005 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:27:44,117 : INFO : EPOCH 44 - PROGRESS: at 46.42% examples, 123764 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:27:45,162 : INFO : EPOCH 44 - PROGRESS: at 48.99% examples, 124083 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:27:46,253 : INFO : EPOCH 44 - PROGRESS: at 51.52% examples, 123867 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:27:47,374 : INFO : EPOCH 44 - PROGRESS: at 55.16% examples, 123902 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:27:48,405 : INFO : EPOCH 44 - PROGRESS: at 58.50% examples, 124308 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:27:49,452 : INFO : EPOCH 44 - PROGRESS: at 61.34% examples, 124265 words/s

2020-05-04 21:28:52,917 : INFO : EPOCH 46 - PROGRESS: at 24.87% examples, 116574 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:28:53,934 : INFO : EPOCH 46 - PROGRESS: at 27.87% examples, 117902 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:28:54,979 : INFO : EPOCH 46 - PROGRESS: at 30.74% examples, 118406 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:28:56,039 : INFO : EPOCH 46 - PROGRESS: at 33.23% examples, 118991 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:28:57,116 : INFO : EPOCH 46 - PROGRESS: at 36.47% examples, 119443 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:28:58,129 : INFO : EPOCH 46 - PROGRESS: at 39.17% examples, 119899 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:28:59,193 : INFO : EPOCH 46 - PROGRESS: at 42.61% examples, 120297 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:29:00,218 : INFO : EPOCH 46 - PROGRESS: at 45.38% examples, 120919 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:29:01,243 : INFO : EPOCH 46 - PROGRESS: at 48.18% examples, 121010 words/s

2020-05-04 21:30:03,768 : INFO : EPOCH 48 - PROGRESS: at 8.84% examples, 124164 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:30:04,777 : INFO : EPOCH 48 - PROGRESS: at 12.01% examples, 125633 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:30:05,798 : INFO : EPOCH 48 - PROGRESS: at 14.88% examples, 124678 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:30:06,831 : INFO : EPOCH 48 - PROGRESS: at 17.51% examples, 125625 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:30:07,854 : INFO : EPOCH 48 - PROGRESS: at 20.48% examples, 125681 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:30:08,863 : INFO : EPOCH 48 - PROGRESS: at 23.38% examples, 125916 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:30:09,911 : INFO : EPOCH 48 - PROGRESS: at 26.32% examples, 125713 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:30:10,911 : INFO : EPOCH 48 - PROGRESS: at 29.18% examples, 125526 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:30:11,944 : INFO : EPOCH 48 - PROGRESS: at 31.75% examples, 125506 words/s,

2020-05-04 21:31:14,467 : INFO : EPOCH 50 - PROGRESS: at 3.27% examples, 125176 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:31:15,486 : INFO : EPOCH 50 - PROGRESS: at 6.31% examples, 121310 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:31:16,556 : INFO : EPOCH 50 - PROGRESS: at 8.47% examples, 119316 words/s, in_qsize 3, out_qsize 1
2020-05-04 21:31:17,586 : INFO : EPOCH 50 - PROGRESS: at 11.44% examples, 119749 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:31:18,626 : INFO : EPOCH 50 - PROGRESS: at 14.41% examples, 120578 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:31:19,636 : INFO : EPOCH 50 - PROGRESS: at 16.84% examples, 120134 words/s, in_qsize 3, out_qsize 0
2020-05-04 21:31:20,714 : INFO : EPOCH 50 - PROGRESS: at 19.77% examples, 120386 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:31:21,745 : INFO : EPOCH 50 - PROGRESS: at 22.77% examples, 121356 words/s, in_qsize 4, out_qsize 0
2020-05-04 21:31:22,752 : INFO : EPOCH 50 - PROGRESS: at 25.64% examples, 121748 words/s, i

In [19]:
model.save(saved_path)

2020-05-04 21:33:05,825 : INFO : saving Doc2Vec object under /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_dmpv.bin, separately None
2020-05-04 21:33:05,827 : INFO : storing np array 'syn1neg' to /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_dmpv.bin.trainables.syn1neg.npy
2020-05-04 21:33:12,992 : INFO : saved /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_dmpv.bin


In [20]:
result=[]
for query in queries:
    rank=model.docvecs.most_similar(positive=[model.infer_vector(query,alpha=0.01,steps=1000)],topn=2914)
    result.append(rank)

2020-05-04 21:33:25,191 : INFO : precomputing L2-norms of doc weight vectors


In [21]:
result[10]

[('C1033', 0.6872710585594177),
 ('C175', 0.6623642444610596),
 ('C2142', 0.6545938849449158),
 ('C1642', 0.6458711624145508),
 ('C737', 0.642140805721283),
 ('C577', 0.6393578052520752),
 ('AILA11', 0.6283252239227295),
 ('C2009', 0.6259872317314148),
 ('C1460', 0.6225141286849976),
 ('C272', 0.6151916980743408),
 ('C2164', 0.6095486283302307),
 ('C1182', 0.5974118709564209),
 ('C2587', 0.5971968173980713),
 ('C360', 0.5968708395957947),
 ('C1604', 0.5957704186439514),
 ('C2186', 0.5949002504348755),
 ('C73', 0.5939537286758423),
 ('C1260', 0.5933749675750732),
 ('C748', 0.5929948091506958),
 ('C91', 0.5884591341018677),
 ('C1120', 0.5842121839523315),
 ('C2787', 0.5785908699035645),
 ('C1354', 0.5784905552864075),
 ('C376', 0.5741114020347595),
 ('C136', 0.5689936280250549),
 ('C498', 0.5660253763198853),
 ('C1946', 0.5614094734191895),
 ('C2643', 0.5606071949005127),
 ('C1786', 0.5564349889755249),
 ('C2398', 0.5559104681015015),
 ('C2734', 0.5536732077598572),
 ('C481', 0.552621245

In [22]:
file=open(output_file,'w')
for i in range(11,51):
    for j in range(len(result[i-1])):
        line="AILA_Q"+str(i)+" "+"Q0 "+str(result[i-1][j][0])+" "+str(j+1)+" "+str(result[i-1][j][1])
        file.write(line)
        file.write("\n")
file.close()

In [113]:
del model

## 5. Doc2Vec DBOW

In [23]:
Documents_train=Documents+queries[0:10]
Documents_train = [TaggedDocument(doc, [label]) for label, doc in zip(labels,Documents_train)]

In [117]:
saved_path = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_dbow_1may_1.bin" #output model
#pretrained_emb = "/home/spaggy/Documents/NLP Project 2020/Doc2Vec/law2vec embeddings/Law2Vec.200d.txt" #pretrained word embeddings
output_file="/home/spaggy/Documents/NLP Project 2020/AILA-dataset/Evaluation_061/evalutaion_file_dbow_1may_1.txt" #Predicted output

In [115]:
#doc2vec parameters
vector_size = 100
window_size = 20
min_count = 1
sampling_threshold = 1e-6
negative_size = 5
train_epoch =50
dm = 0 #0 = dbow; 1 = dmpv
worker_count = 4 #number of parallel processes


In [116]:
#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#train doc2vec model
#docs = g.doc2vec.TaggedLineDocument(train_corpus)
#model = g.Doc2Vec(Documents, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)
model = g.Doc2Vec(Documents_train, size=vector_size, window=window_size, min_count=min_count, workers=worker_count,dm=dm, iter=train_epoch)


2020-05-01 21:29:54,976 : INFO : collecting all words and their counts
2020-05-01 21:29:54,978 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-05-01 21:29:55,812 : INFO : collected 56484 word types and 2924 unique tags from a corpus of 2924 examples and 4860647 words
2020-05-01 21:29:55,813 : INFO : Loading a fresh vocabulary
2020-05-01 21:29:55,922 : INFO : effective_min_count=1 retains 56484 unique words (100% of original 56484, drops 0)
2020-05-01 21:29:55,924 : INFO : effective_min_count=1 leaves 4860647 word corpus (100% of original 4860647, drops 0)
2020-05-01 21:29:56,086 : INFO : deleting the raw counts dictionary of 56484 items
2020-05-01 21:29:56,090 : INFO : sample=0.001 downsamples 37 most-common words
2020-05-01 21:29:56,091 : INFO : downsampling leaves estimated 4557695 word corpus (93.8% of prior 4860647)
2020-05-01 21:29:56,260 : INFO : estimated required memory for 56484 words and 100 dimensions: 75183600 bytes
2020-05-01 21:29:56,2

2020-05-01 21:30:29,354 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-01 21:30:29,362 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-01 21:30:29,366 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-01 21:30:29,370 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-01 21:30:29,372 : INFO : EPOCH - 9 : training on 4860647 raw words (4502377 effective words) took 3.5s, 1303898 effective words/s
2020-05-01 21:30:30,377 : INFO : EPOCH 10 - PROGRESS: at 29.75% examples, 1291677 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:30:31,381 : INFO : EPOCH 10 - PROGRESS: at 58.86% examples, 1293566 words/s, in_qsize 7, out_qsize 0
2020-05-01 21:30:32,385 : INFO : EPOCH 10 - PROGRESS: at 87.11% examples, 1306110 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:30:32,800 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-01 21:30:32,816 : INFO : worker thread finished; await

2020-05-01 21:31:02,288 : INFO : EPOCH 19 - PROGRESS: at 60.53% examples, 1323012 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:31:03,289 : INFO : EPOCH 19 - PROGRESS: at 87.76% examples, 1322406 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:31:03,674 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-01 21:31:03,677 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-01 21:31:03,682 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-01 21:31:03,687 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-01 21:31:03,691 : INFO : EPOCH - 19 : training on 4860647 raw words (4502071 effective words) took 3.4s, 1320858 effective words/s
2020-05-01 21:31:04,711 : INFO : EPOCH 20 - PROGRESS: at 30.98% examples, 1330722 words/s, in_qsize 7, out_qsize 0
2020-05-01 21:31:05,716 : INFO : EPOCH 20 - PROGRESS: at 61.73% examples, 1341875 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:31:06,724 : INFO : EPOC

2020-05-01 21:31:34,875 : INFO : EPOCH - 28 : training on 4860647 raw words (4502357 effective words) took 3.5s, 1297490 effective words/s
2020-05-01 21:31:35,884 : INFO : EPOCH 29 - PROGRESS: at 30.13% examples, 1302734 words/s, in_qsize 7, out_qsize 0
2020-05-01 21:31:36,889 : INFO : EPOCH 29 - PROGRESS: at 61.01% examples, 1329783 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:31:37,890 : INFO : EPOCH 29 - PROGRESS: at 88.95% examples, 1341482 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:31:38,223 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-01 21:31:38,229 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-01 21:31:38,234 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-01 21:31:38,239 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-01 21:31:38,244 : INFO : EPOCH - 29 : training on 4860647 raw words (4502170 effective words) took 3.4s, 1337987 effective words/s
2020-05-01 21

2020-05-01 21:32:09,958 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-01 21:32:09,964 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-01 21:32:09,968 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-01 21:32:09,972 : INFO : EPOCH - 38 : training on 4860647 raw words (4502799 effective words) took 3.4s, 1309030 effective words/s
2020-05-01 21:32:10,985 : INFO : EPOCH 39 - PROGRESS: at 25.41% examples, 1098861 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:32:11,986 : INFO : EPOCH 39 - PROGRESS: at 54.24% examples, 1195106 words/s, in_qsize 8, out_qsize 1
2020-05-01 21:32:12,987 : INFO : EPOCH 39 - PROGRESS: at 83.04% examples, 1242867 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:32:13,573 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-01 21:32:13,575 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-01 21:32:13,584 : INFO : worker thread finished; awai

2020-05-01 21:32:44,230 : INFO : EPOCH 48 - PROGRESS: at 85.26% examples, 1277506 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:32:44,745 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-01 21:32:44,753 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-01 21:32:44,754 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-01 21:32:44,758 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-01 21:32:44,759 : INFO : EPOCH - 48 : training on 4860647 raw words (4501994 effective words) took 3.5s, 1270876 effective words/s
2020-05-01 21:32:45,775 : INFO : EPOCH 49 - PROGRESS: at 28.49% examples, 1232365 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:32:46,779 : INFO : EPOCH 49 - PROGRESS: at 56.77% examples, 1246730 words/s, in_qsize 7, out_qsize 0
2020-05-01 21:32:47,781 : INFO : EPOCH 49 - PROGRESS: at 82.90% examples, 1238496 words/s, in_qsize 8, out_qsize 0
2020-05-01 21:32:48,646 : INFO : work

In [118]:
model.save(saved_path)

2020-05-01 21:33:11,568 : INFO : saving Doc2Vec object under /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_dbow_1may_1.bin, separately None
2020-05-01 21:33:12,248 : INFO : saved /home/spaggy/Documents/NLP Project 2020/Doc2Vec/test models/model_dbow_1may_1.bin


In [119]:
result=[]
for query in queries:
    rank=model.docvecs.most_similar(positive=[model.infer_vector(query,alpha=0.01,steps=1000)],topn=2914)
    result.append(rank)

2020-05-01 21:34:22,448 : INFO : precomputing L2-norms of doc weight vectors


In [120]:
result[10]

[('C1354', 0.6721161007881165),
 ('C2158', 0.663282573223114),
 ('C2787', 0.6574461460113525),
 ('C175', 0.6257264614105225),
 ('C161', 0.6091955900192261),
 ('C2017', 0.6055167317390442),
 ('C418', 0.5907808542251587),
 ('C2506', 0.5883035063743591),
 ('C2389', 0.5758793950080872),
 ('C2643', 0.5690625905990601),
 ('C1731', 0.5688117146492004),
 ('C2351', 0.5609655380249023),
 ('C2852', 0.5550179481506348),
 ('C73', 0.5541611313819885),
 ('C1457', 0.5492089986801147),
 ('C1332', 0.5489586591720581),
 ('C262', 0.5485392808914185),
 ('C2257', 0.5437439680099487),
 ('C91', 0.5435388088226318),
 ('C913', 0.5433363318443298),
 ('C1095', 0.542548418045044),
 ('C272', 0.539557933807373),
 ('C730', 0.5354180335998535),
 ('C1642', 0.532139778137207),
 ('C1916', 0.5314419269561768),
 ('C2164', 0.5277328491210938),
 ('C1824', 0.5266416072845459),
 ('C1547', 0.5253700017929077),
 ('C2448', 0.5187736749649048),
 ('C1235', 0.5187320709228516),
 ('C737', 0.5168018341064453),
 ('C1531', 0.51656413078

In [121]:
file=open(output_file,'w')
for i in range(11,51):
    for j in range(len(result[i-1])):
        line="AILA_Q"+str(i)+" "+"Q0 "+str(result[i-1][j][0])+" "+str(j+1)+" "+str(result[i-1][j][1])
        file.write(line)
        file.write("\n")
file.close()